In [ ]:
import numpy as np

In [1]:
modelnames = ['Vertically Offset\nStandard Conductance Based Formulation',
              'Vertically Offset\nXT3D',
              'Vertically Staggered\nStandard Conductance Based Formulation',
              'Vertically Staggered\nXT3D']

In [ ]:
def fig2old(vmin, vmax, extent, scale_chan, scale_dom, normalize):    
    fig, axs = plt.subplots(2, 2, figsize = (6,6))
    for i, ax in enumerate(fig.axes):
        
        model, head, spdismf6, qx, qy, qz = models_all[i], head_all[i], spdismf6_all[i], qx_all[i], qy_all[i], qz_all[i]
        
        qx_chan = np.ma.masked_where(icelltype != 1, qx)
        qy_chan = np.ma.masked_where(icelltype != 1, qy)
        qz_chan = np.ma.masked_where(icelltype != 1, qz)
        qx_dom = np.ma.masked_where(icelltype != 0, qx)
        qy_dom = np.ma.masked_where(icelltype != 0, qy)
        qz_dom = np.ma.masked_where(icelltype != 0, qz)
        
        ax = plt.subplot(2,2,i+1,aspect="equal")
        ax.set_title(modelnames[i], size = 10)
        xs = flopy.plot.PlotCrossSection(model=model, line={"line": [(0, 0.5),(Lx, 0.5)]}, extent=extent,)
        cb = xs.plot_array(a = head, cmap = 'jet', alpha=0.8, vmin = vmin, vmax = vmax)
        xs.plot_vector(qx_chan, qy_chan, qz_chan, scale=scale_chan, normalize=normalize, color="black")
        xs.plot_vector(qx_dom, qy_dom, qz_dom, scale=scale_dom, normalize=normalize, color="gray")
        xs.plot_grid(lw = 0.1, color = 'black')
        
        ax.set_xlabel('x (m)', size = 10)
        if i == 0: ax.set_ylabel('z (m)', size = 10)
        
        plt.colorbar(cb, shrink = 0.3)
    plt.tight_layout()  
    plt.savefig('fig2.tif', dpi=450)

In [ ]:
def fig2(i, vmin, vmax, extent, scale_chan, scale_bndy, scale_dom, normalize):    
    
    fig, axs = plt.subplots(1, 2, figsize = (10,6), constrained_layout=True)
    fig.suptitle(modelnames[i], fontsize=16)
    fig2cell(fig, i, vmin, vmax, extent, scale_chan, scale_dom, normalize)
    fig2face(fig, i, vmin, vmax, extent, scale_chan, scale_bndy, scale_dom, normalize)

In [ ]:
def fig2cell(fig, i, vmin, vmax, extent, scale_chan, scale_dom, normalize):    

    qx_chan = np.ma.masked_where(icelltype != 1, qx)
    qy_chan = np.ma.masked_where(icelltype != 1, qy)
    qz_chan = np.ma.masked_where(icelltype != 1, qz)
    qx_dom = np.ma.masked_where(icelltype != 0, qx)
    qy_dom = np.ma.masked_where(icelltype != 0, qy)
    qz_dom = np.ma.masked_where(icelltype != 0, qz)

    ax = plt.subplot(1,2,1,aspect="equal")
    if (spdis_recalc):
        titletext = "specific discharge (custom)"
    else:
        titletext = "specific discharge (mf6)"
    ax.set_title(titletext, size = 12)
    xs = flopy.plot.PlotCrossSection(model=gwf, line={"line": [(0, 0.5),(Lx, 0.5)]}, extent=extent,)
    cbct = xs.plot_array(a = icelltype, cmap = 'gray', alpha=0.4, vmin = -4, vmax = 1)
    #cb = xs.plot_array(a = head, cmap = 'jet', alpha=0.8, vmin = vmin, vmax = vmax)
    cb = xs.contour_array(a = head, cmap='Oranges', alpha=0.8, vmin = vmin, vmax = vmax)
    xs.plot_vector(qx_chan, qy_chan, qz_chan, scale=scale_chan, normalize=normalize, color="black")
    xs.plot_vector(qx_dom, qy_dom, qz_dom, scale=scale_dom, normalize=normalize, color="gray")
    xs.plot_grid(lw = 0.1, color = 'black')

    ax.set_xlabel('x (m)', size = 10)
    if i == 0: ax.set_ylabel('z (m)', size = 10)

    #plt.colorbar(cb, shrink = 0.3)
    #plt.tight_layout()  
    #plt.savefig('fig2test.tif', dpi=450)

In [ ]:
def fig2face(fig, i, vmin, vmax, extent, scale_chan, scale_bndy, scale_dom, normalize):
        
    gp = d2d.get_gridprops_disu6()
    nja = gp["nja"]
    ihc = gp["ihc"]

    qxface = np.zeros((nja))
    qzface = np.zeros((nja))
    for iconn in range(nja):
        if ihc[iconn] > 0:
            qxface[iconn] = qnface[iconn]
        else:
            qzface[iconn] = qnface[iconn]

    ax = plt.subplot(1,2,2,aspect="equal")
    ax.set_title("face fluxes", size = 12)
    xs = flopy.plot.PlotCrossSection(model=gwf, line={"line": [(0, 0.5),(Lx, 0.5)]}, extent=extent,)
    cbct = xs.plot_array(a = icelltype, cmap = 'gray', alpha=0.4, vmin = -4, vmax = 1)
    #cb = xs.contour_array(a = head, cmap='Purples', alpha=0.4, vmin = vmin, vmax = vmax)
    qxplot = np.ma.masked_where(ifacetype != 0, qxface)
    qzplot = np.ma.masked_where(ifacetype != 0, qzface)
    plt.quiver(xface, zface, qxplot, qzplot, scale=scale_dom, pivot='mid', color='blue')
    qxplot = np.ma.masked_where(ifacetype != 1, qxface)
    qzplot = np.ma.masked_where(ifacetype != 1, qzface)
    plt.quiver(xface, zface, qxplot, qzplot, scale=scale_bndy, pivot='mid', color='green')
    qxplot = np.ma.masked_where(ifacetype != 2, qxface)
    qzplot = np.ma.masked_where(ifacetype != 2, qzface)
    plt.quiver(xface, zface, qxplot, qzplot, scale=scale_chan, pivot='mid', color='red')
    xs.plot_grid(lw = 0.1, color = 'gray')

    ax.set_xlabel('x (m)', size = 10)
    if i == 0: ax.set_ylabel('z (m)', size = 10)

    #plt.colorbar(cb, shrink = 0.3)
    #plt.tight_layout()  
    #plt.savefig('fig2face.tif', dpi=450)

In [2]:
def fig3():
    
    fig, axes = plt.subplots(1,2, figsize=(16, 3))

    ax = plt.subplot(121)
    ax.set_title('Flux magnitude with refinement\n') 
    ax.set_ylim([0.5,1.2])
    ax.set_xlabel('Model layers per hydrogeologic layer')
    ax.set_ylabel('Flux (m/d)')

    lay_per_chan, q_mag, q_angle = list(zip(*vo_results))
    ax.plot(lay_per_chan, q_mag, marker = 'o', linestyle = '-') # VO
    lay_per_chan, q_mag, q_angle = list(zip(*vs_results))
    ax.plot(lay_per_chan, q_mag, marker = 'o', linestyle = '-') # VS
    ax.plot(lay_per_chan, np.ones(len(lay_per_chan)), linestyle = '--', color = 'black') # Analytical
    ax.legend(['Vertically Offset', 'Vertically Staggered', 'Analytical'])
    ax.set_xticks(resolution_scenarios)

    ax = plt.subplot(122)
    ax.set_title('Flux angle with refinement\n') 
    ax.set_ylim([0,35])
    ax.set_xlabel('Model layers per hydrogeologic layer')
    ax.set_ylabel('theta (degrees)')

    lay_per_chan, q_mag, q_angle = list(zip(*vo_results)) # VO
    ax.plot(lay_per_chan, q_angle, marker = 'o', linestyle = '-')
    lay_per_chan, q_mag, q_angle = list(zip(*vs_results)) # VS
    ax.plot(lay_per_chan, q_angle, marker = 'o', linestyle = '-') 
    ax.plot(lay_per_chan, 30 * np.ones(len(lay_per_chan)), linestyle = '--', color = 'black') # Analytical
    ax.legend(['Vertically Offset', 'Vertically Staggered','Analytical'])
    ax.set_xticks(resolution_scenarios)

    plt.show()

In [3]:
def fig4alt2():    
    
    x = dip
    #logfunc = lambda e: np.log10(e)
    #y = logfunc(Kcontrast)
    
    for s in range(4): # Four scenarios
               
        fig, axs = plt.subplots(1, 2, figsize = (10,3))
        fig.suptitle = modelnames[s]
        
        # FLUX MAG
        ax = plt.subplot(121)
        for kc, kcval in enumerate(Kcontrast):
            z = qmag_results[s][kc]
            ax.plot(x, z, marker = None, linestyle = '-', label = kcval)
            ax.set_title('Flux magnitude')
            ax.set_xlabel('Dip (degrees)')
            ax.legend()
        ax.plot(x, np.ones(len(x)), linestyle = '--', color = 'black') # Analytical

        # FLUX ANGLE
        ax = plt.subplot(122)
        for kc, kcval in enumerate(Kcontrast):
            z = qang_results[s][kc]
            ax.plot(x, z, marker = None, linestyle = '-', label = kcval)
            ax.set_title('Flux direction')
            ax.set_xlabel('Dip (degrees)')
            ax.legend()
        ax.plot(x, x, linestyle = '--', color = 'black') # Analytical

        plt.tight_layout()
        plt.show()
        figname = 'fig3alt2_' + str(s) + '.png'
        plt.savefig(figname, dpi=450)

In [3]:
def fig4alt():    
    
    x = dip
    #logfunc = lambda e: np.log10(e)
    #y = logfunc(Kcontrast)
    
    for s in range(4): # Four scenarios
               
        fig, axs = plt.subplots(1, 2, figsize = (10,3))
        fig.suptitle = modelnames[s]
        
        # FLUX MAG
        ax = plt.subplot(121)
        for kc, kcval in enumerate(Kcontrast):
            z = abs(qmag_results[s][kc] - 1)*100
            ax.plot(x, z, marker = None, linestyle = '-', label = kcval)
            ax.set_title('Flux magnitude percentage error')
            ax.set_xlabel('Dip (degrees)')
            ax.legend()

        # FLUX ANGLE
        ax = plt.subplot(122)
        for kc, kcval in enumerate(Kcontrast):
            z = abs((qang_results[s][kc] - dip)/dip)*100
            ax.plot(x, z, marker = None, linestyle = '-', label = kcval)
            ax.set_title('Flux direction percentage error')
            ax.set_xlabel('Dip (degrees)')
            ax.legend()

        plt.tight_layout()
        plt.show()
        figname = 'fig3alt_' + str(s) + '.png'
        plt.savefig(figname, dpi=450)

In [3]:
def fig4():    
    
    levels = [0,10,20,30,40,50,60,70,80,90,100,1000]
    
    for s in range(4): # Four scenarios
               
        fig, axs = plt.subplots(1, 2, figsize = (10,3))
        fig.suptitle = modelnames[s]
        
        #for i, ax in enumerate(fig.axes):
        if True:
            x = dip
            logfunc = lambda e: np.log10(e)
            y = logfunc(Kcontrast)

            # FLUX MAG
            ax = plt.subplot(121)
            ax.set_title('Flux magnitude percentage error')
            z = abs(qmag_results[s] - 1)*100

            contourf_ = ax.contourf(x,y,z, levels, vmin = 0, vmax = 100, cmap = 'bwr', alpha = 0.6)
            ax.contour(x,y,z, levels, colors = 'Black', linewidths=0.5, linestyles = 'solid')
            cbar = fig.colorbar(contourf_) 
            ax.set_xlabel('Dip (degrees)')
            ax.set_ylabel('K contrast (log)')

            # FLUX ANGLE
            ax = plt.subplot(122)
            ax.set_title('Flux direction percentage error')
            ax.set_xlabel('Dip (degrees)')

            z = np.zeros((len(Kcontrast), len(dip)))
            for kc, qang in enumerate(qang_results[s]):
                z[kc] = abs((qang - dip)/dip)*100
            contourf_ = ax.contourf(x,y,z, levels, vmin = 0, vmax = 100, cmap = 'bwr', alpha = 0.6)
            ax.contour(x,y,z, levels, colors = 'Black', linewidths=0.5, linestyles = 'solid')
            cbar = fig.colorbar(contourf_) 
        plt.tight_layout()
        plt.show()
        figname = 'fig3_' + str(s) + '.png'
        plt.savefig(figname, dpi=450)

In [4]:
def table1():
    for i in range(4):
        print(modelnames[i])
        print('qmagmid = ', q_all[i][0])
        print('qangmid = ', q_all[i][1], '\n')
        
        #print ("\nSPECIFIC DISCHARGE:")
        #qdata = [
        #    ["analytical", qmagana, qangana],
        #    ["mid-channel", qmagmid, qangmid],
        #    ["avg channel", qmagavg, qangavg],
        #    ["min channel", qmagmin, qangmin],
        #    ["max channel", qmagmax, qangmax],
        #]
        #qhead = ["", "magnitude", "angle",]
        #print(tabulate(qdata, headers=qhead, tablefmt="simple", floatfmt=".14f"))